In [1]:
import mindspore as ms
from mindspore import nn
from mindspore.dataset import ImageFolderDataset
from mindspore.dataset import transforms
from mindspore.dataset import vision
# from mindvision.classification.models import resnet50
import network as nw

In [2]:
# 加载数据集
def data_load():
    # class_indexing = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
    class_indexing = {'冰激凌': 0,
                      '鸡蛋布丁': 1,
                      '烤冷面': 2,
                      '芒果班戟': 3,
                      '三明治': 4,
                      '松鼠鱼': 5,
                      '甜甜圈': 6,
                      '土豆泥': 7,
                      '小米粥': 8,
                      '玉米饼': 9}
    data_train = ImageFolderDataset(dataset_dir='./mindcon_food_classification/train', class_indexing=class_indexing,shuffle=True,decode=True)
    # data_test = ImageFolderDataset(dataset_dir='./mindcon_food_classification/evalu', class_indexing=class_indexing,
                                    # shuffle=True, decode=True)
    return data_train

In [3]:
# 数据集转换
def data_transform(data_train):
    composed = transforms.c_transforms.Compose(
        [
            vision.c_transforms.Resize([128,128]),#缩放
            vision.c_transforms.Rescale(1.0 / 255.0, 0),
            vision.c_transforms.HWC2CHW()
        ]
    )
    data_train1 = data_train.map(composed, 'image')
    return data_train1

In [4]:
# 加载网络
def load_network():
    network = resnet50(pretrained=True)
    head = nn.SequentialCell([nn.Dropout(0.6),
                              nn.Dense(2048, 10)])
    network.head=head
    return network

In [4]:
# 加载网络自创
def load_network2():
    network= nw.network()
    return network

In [6]:
def network():
    network = nn.SequentialCell([
        nn.Conv2d(3, 64, 3),
        nn.ReLU(),
        # nn.AdaptiveMaxPool2d(128),
        nn.MaxPool2d(3,3),

        nn.Conv2d(64, 64, 3),
        nn.ReLU(),
        # nn.AdaptiveMaxPool2d(64),
        nn.MaxPool2d(3,3),

        nn.Flatten(),
        nn.Dense(64 * 14 * 14, 512),
        nn.ReLU(),
        # nn.BatchNorm1d(512),
        # nn.Dropout(0.8),
        nn.Dense(512, 512),
        nn.ReLU(),
        # nn.BatchNorm1d(512),
        # nn.Dropout(0.8),
        nn.Dense(512, 10),
        # nn.Softmax()
    ])
    return network

In [5]:
if __name__ == '__main__':
    data_train = data_load()
    data_train = data_transform(data_train)
    # data_test = data_transform(data_test)
    data_train = data_train.batch(batch_size=128)
    # data_test = data_test.batch(batch_size=32)

    # 网络构建
    network=load_network2()
    # 损失函数
    loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
    # loss_fn = nn.BCELoss()
    # 优化器
    optimizer = nn.SGD(network.trainable_params(), learning_rate=0.0001)
    # 回调函数--保存
    ckpoint_cb = ms.train.callback.ModelCheckpoint(prefix='food', directory='./checkpoint', config=ms.train.callback.CheckpointConfig(save_checkpoint_steps=300))
    # 加载模型参数
    # load_model()
    # 模型构建
    model = ms.Model(network=network,loss_fn=loss_fn,optimizer=optimizer,metrics={'accuracy'})
    # 训练
    model.train(100,data_train,callbacks=[ms.train.callback.LossMonitor()])
    #评估
    # print(model.eval(data_test))
    # print(i)
    #保存模型
    ms.save_checkpoint(network, "./checkpoint/MyNet.ckpt")

epoch: 1 step: 192, loss is 1.4640583992004395
epoch: 2 step: 192, loss is 1.2650094032287598
epoch: 3 step: 192, loss is 0.9686372876167297
epoch: 4 step: 192, loss is 0.9155887961387634
epoch: 5 step: 192, loss is 0.6024640202522278
epoch: 6 step: 192, loss is 0.6743699312210083
epoch: 7 step: 192, loss is 0.6666436195373535
epoch: 8 step: 192, loss is 0.5020627975463867
epoch: 9 step: 192, loss is 0.3640795946121216
epoch: 10 step: 192, loss is 0.4098779261112213
epoch: 11 step: 192, loss is 0.29544252157211304
epoch: 12 step: 192, loss is 0.2667683959007263
epoch: 13 step: 192, loss is 0.2635415196418762
epoch: 14 step: 192, loss is 0.26624929904937744
epoch: 15 step: 192, loss is 0.2524968683719635
epoch: 16 step: 192, loss is 0.1696675568819046
epoch: 17 step: 192, loss is 0.18507060408592224
epoch: 18 step: 192, loss is 0.11080233752727509
epoch: 19 step: 192, loss is 0.3015005886554718
epoch: 20 step: 192, loss is 0.11018410325050354
epoch: 21 step: 192, loss is 0.0832041129469